In [68]:
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn import grid_search
from sklearn.grid_search import ParameterGrid

import util

In [2]:
def extract_feats(ffs, direc="train", global_feat_dict=None):
    """
    arguments:
      ffs are a list of feature-functions.
      direc is a directory containing xml files (expected to be train or test).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.

    returns: 
      a sparse design matrix, a dict mapping features to column-numbers,
      a vector of target classes, and a list of system-call-history ids in order 
      of their rows in the design matrix.
      
      Note: the vector of target classes returned will contain the true indices of the
      target classes on the training data, but will contain only -1's on the test
      data
    """
    fds = [] # list of feature dicts
    classes = []
    ids = [] 
    for datafile in os.listdir(direc):
        # extract id and true class (if available) from filename
        id_str,clazz = datafile.split('.')[:2]
        ids.append(id_str)
        # add target class if this is training data
        try:
            classes.append(util.malware_classes.index(clazz))
        except ValueError:
            # we should only fail to find the label in our list of malware classes
            # if this is test data, which always has an "X" label
            assert clazz == "X"
            classes.append(-1)
        rowfd = {}
        # parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
        # accumulate features
        [rowfd.update(ff(tree)) for ff in ffs]
        fds.append(rowfd)
        
    X,feat_dict = make_design_mat(fds,global_feat_dict)
    return X, feat_dict, np.array(classes), ids


def make_design_mat(fds, global_feat_dict=None):
    """
    arguments:
      fds is a list of feature dicts (one for each row).
      global_feat_dict is a dictionary mapping feature_names to column-numbers; it
      should only be provided when extracting features from test data, so that 
      the columns of the test matrix align correctly.
       
    returns: 
        a sparse NxD design matrix, where N == len(fds) and D is the number of
        the union of features defined in any of the fds 
    """
    if global_feat_dict is None:
        all_feats = set()
        [all_feats.update(fd.keys()) for fd in fds]
        feat_dict = dict([(feat, i) for i, feat in enumerate(sorted(all_feats))])
    else:
        feat_dict = global_feat_dict
        
    cols = []
    rows = []
    data = []        
    for i in xrange(len(fds)):
        temp_cols = []
        temp_data = []
        for feat,val in fds[i].iteritems():
            try:
                # update temp_cols iff update temp_data
                temp_cols.append(feat_dict[feat])
                temp_data.append(val)
            except KeyError as ex:
                if global_feat_dict is not None:
                    pass  # new feature in test data; nbd
                else:
                    raise ex

        # all fd's features in the same row
        k = len(temp_cols)
        cols.extend(temp_cols)
        data.extend(temp_data)
        rows.extend([i]*k)

    assert len(cols) == len(rows) and len(rows) == len(data)
   

    X = sparse.csr_matrix((np.array(data),
                   (np.array(rows), np.array(cols))),
                   shape=(len(fds), len(feat_dict)))
    return X, feat_dict
    

## Here are two example feature-functions. They each take an xml.etree.ElementTree object, 
# (i.e., the result of parsing an xml file) and returns a dictionary mapping 
# feature-names to numeric values.
## TODO: modify these functions, and/or add new ones.
def first_last_system_call_feats(tree):
    """
    arguments:
      tree is an xml.etree.ElementTree object
    returns:
      a dictionary mapping 'first_call-x' to 1 if x was the first system call
      made, and 'last_call-y' to 1 if y was the last system call made. 
      (in other words, it returns a dictionary indicating what the first and 
      last system calls made by an executable were.)
    """
    c = Counter()
    in_all_section = False
    first = True # is this the first system call
    last_call = None # keep track of last call we've seen
    for el in tree.iter():
        # ignore everything outside the "all_section" element
        if el.tag == "all_section" and not in_all_section:
            in_all_section = True
        elif el.tag == "all_section" and in_all_section:
            in_all_section = False
        elif in_all_section:
            if first:
                c["first_call-"+el.tag] = 1
                first = False
            last_call = el.tag  # update last call seen
            
    # finally, mark last call seen
    c["last_call-"+last_call] = 1
    return c

def system_call_count_feats(tree):
    """
    arguments:
      tree is an xml.etree.ElementTree object
    returns:
      a dictionary mapping 'num_system_calls' to the number of system_calls
      made by an executable (summed over all processes)
    """
    c = Counter()
    in_all_section = False
    for el in tree.iter():
        # ignore everything outside the "all_section" element
        if el.tag == "all_section" and not in_all_section:
            in_all_section = True
        elif el.tag == "all_section" and in_all_section:
            in_all_section = False
        elif in_all_section:
#             c['num_system_calls'] += 1
            if el.tag not in c:
                c[el.tag] = 0
            else:
                c[el.tag] += 1
    return c

"""
## The following function does the feature extraction, learning, and prediction
def main():
    train_dir = "train"
    test_dir = "test"
    outputfile = "sample_predictions.csv"  # feel free to change this or take it as an argument
    
    # TODO put the names of the feature functions you've defined above in this list
    ffs = [first_last_system_call_feats, system_call_count_feats]
    
    # extract features
    print "extracting training features..."
    X_train,global_feat_dict,t_train,train_ids = extract_feats(ffs, train_dir)
    print "done extracting training features"
    print
    
    # TODO train here, and learn your classification parameters
    print "learning..."
    learned_W = np.random.random((len(global_feat_dict),len(util.malware_classes)))
    print "done learning"
    print
    
    # get rid of training data and load test data
    del X_train
    del t_train
    del train_ids
    print "extracting test features..."
    X_test,_,t_ignore,test_ids = extract_feats(ffs, test_dir, global_feat_dict=global_feat_dict)
    print "done extracting test features"
    print
    
    # TODO make predictions on text data and write them out
    print "making predictions..."
    preds = np.argmax(X_test.dot(learned_W),axis=1)
    print "done making predictions"
    print
    
    print "writing predictions..."
    util.write_predictions(preds, test_ids, outputfile)
    print "done!"
"""

'\n## The following function does the feature extraction, learning, and prediction\ndef main():\n    train_dir = "train"\n    test_dir = "test"\n    outputfile = "sample_predictions.csv"  # feel free to change this or take it as an argument\n    \n    # TODO put the names of the feature functions you\'ve defined above in this list\n    ffs = [first_last_system_call_feats, system_call_count_feats]\n    \n    # extract features\n    print "extracting training features..."\n    X_train,global_feat_dict,t_train,train_ids = extract_feats(ffs, train_dir)\n    print "done extracting training features"\n    print\n    \n    # TODO train here, and learn your classification parameters\n    print "learning..."\n    learned_W = np.random.random((len(global_feat_dict),len(util.malware_classes)))\n    print "done learning"\n    print\n    \n    # get rid of training data and load test data\n    del X_train\n    del t_train\n    del train_ids\n    print "extracting test features..."\n    X_test,_,t_i

In [3]:
"""
if __name__ == "__main__":
    main()
"""

'\nif __name__ == "__main__":\n    main()\n'

In [4]:
train_dir = "train"
test_dir = "test"
outputfile = "sample_predictions.csv"  # feel free to change this or take it as an argument

# TODO put the names of the feature functions you've defined above in this list
ffs = [first_last_system_call_feats, system_call_count_feats]

In [110]:
# extract features
print "extracting training features..."
X_train,global_feat_dict,t_train,train_ids = extract_feats(ffs, train_dir)
print "done extracting training features"
print

extracting training features...
done extracting training features



In [111]:
np.random.seed(9001)
msk = np.random.rand(X_train.shape[0]) < 0.85

X_train, X_valid = X_train[msk], X_train[~msk]
t_train, t_valid = t_train[msk], t_train[~msk]

In [7]:
# TODO train here, and learn your classification parameters
print "learning..."
learned_W = np.random.random((len(global_feat_dict),len(util.malware_classes)))
print "done learning"
print

learning...
done learning



In [98]:
# kNN Model
knn_parameters = {'n_neighbors': range(1,21) + [25, 50, 100, 150], 'weights': ['uniform','distance'], 'p':[1, 2]}
knn = grid_search.GridSearchCV(KNeighborsClassifier(), knn_parameters)
knn.fit(X_train, t_train)

print 'kNN Hyperparameters'
print '  n neighbors:', knn.best_estimator_.n_neighbors
print '  weights:', knn.best_estimator_.weights
print '  Minkowski power:', knn.best_estimator_.p
print '-----'
print 'train score:', knn.score(X_train,t_train)
print 'validations score:', knn.score(X_valid,t_valid)

kNN Hyperparameters
  n neighbors: 17
  weights: distance
  Minkowski power: 1
-----
train score: 0.984779299847793
validations score: 0.8755458515283843


In [99]:
# Random Forest Model
rf_parameters = {'n_estimators': [5, 10, 25, 50, 100, 250, 500, 750, 1000], 'max_features': ['sqrt', 'log2', None], 
                 'max_depth': [2, 4, 10, 25, 50, None], 'class_weight': ['balanced', None]}
rf = grid_search.GridSearchCV(RandomForestClassifier(), rf_parameters)
rf.fit(X_train, t_train)

print 'RF Hyperparameters'
print '  n trees:', rf.best_estimator_.n_estimators
print '  max depth:', rf.best_estimator_.max_depth
print '  max features:', rf.best_estimator_.max_features
print '  class weight:', rf.best_estimator_.class_weight
print '-----'
print 'train score:', rf.score(X_train,t_train)
print 'validation score:', rf.score(X_valid,t_valid)

RF Hyperparameters
  n trees: 1000
  max depth: 50
  max features: sqrt
  class weight: None
-----
train score: 0.984779299847793
validation score: 0.8930131004366813


In [112]:
# Logistic Regression Model
lr_parameters = {'cv': range(3,11), 'fit_intercept':[True, False], 'penalty': ['l2']}
lr = grid_search.GridSearchCV(LogisticRegressionCV(), lr_parameters)
lr.fit(X_train, t_train)

print 'LR Hyperparameters'
print '  k CV folds:', lr.best_estimator_.cv
print '  fit intercept:', lr.best_estimator_.fit_intercept
print '  penalty:', lr.best_estimator_.penalty
print '-----'
print 'train score:', lr.score(X_train,t_train)
print 'validation score:', lr.score(X_valid,t_valid)

LR Hyperparameters
  k CV folds: 4
  fit intercept: True
  penalty: l2
-----
train score: 0.8546423135464232
validation score: 0.8493449781659389


In [113]:
# get rid of training data and load test data
# del X_train
# del t_train
# del train_ids
print "extracting test features..."
X_test,_,t_ignore,test_ids = extract_feats(ffs, test_dir, global_feat_dict=global_feat_dict)
print "done extracting test features"
print

extracting test features...
done extracting test features



In [114]:
# TODO make predictions on text data and write them out
print "making predictions..."
# preds = np.argmax(X_test.dot(learned_W),axis=1)
knn_preds = knn.predict(X_test)
rf_preds = rf.predict(X_test)
lr_preds = lr.predict(X_test)
print "done making predictions"
print

making predictions...
done making predictions



In [115]:
print "writing predictions..."
# util.write_predictions(preds, test_ids, outputfile)
util.write_predictions(knn_preds, test_ids, "knn_predictions.csv")
util.write_predictions(rf_preds, test_ids, "rf_predictions.csv")
util.write_predictions(lr_preds, test_ids, "lr_predictions.csv")
print "done!"

writing predictions...
done!


In [116]:
from sklearn.svm import SVC

svm_parameters = {'C':[.25, .5, 1, 2, 5]}
svm = grid_search.GridSearchCV(SVC(), svm_parameters)
svm.fit(X_train, t_train)

print 'SVM Hyperparameters'
print '  C values:', svm.best_estimator_.C
print '-----'
print 'train score:', svm.score(X_train,t_train)
print 'validation score:', svm.score(X_valid,t_valid)

SVM Hyperparameters
  C values: 2
-----
train score: 0.9600456621004566
validation score: 0.7663755458515283


In [118]:
svm_preds = svm.predict(X_test)
util.write_predictions(svm_preds, test_ids, "svm_predictions.csv")

In [117]:
from sklearn.neural_network import MLPClassifier

nn_parameters = {'activation': ['identity','logistic','tanh','relu'], 'alpha': [.0001, .0001, .001, .01, .1, 1]}
nn = grid_search.GridSearchCV(MLPClassifier(), nn_parameters)
nn.fit(X_train, t_train)

print 'NN Hyperparameters'
print '  activation function:', nn.best_estimator_.activation
print '  loss coefficient (alpha value):', nn.best_estimator_.alpha
print '-----'
print 'train score:', nn.score(X_train,t_train)
print 'validation score:', nn.score(X_valid,t_valid)

/anaconda3/envs/my-rdkit-env/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


NN Hyperparameters
  activation function: logistic
  loss coefficient (alpha value): 0.0001
-----
train score: 0.928462709284627
validation score: 0.8777292576419214


In [119]:
nn_preds = nn.predict(X_test)
util.write_predictions(nn_preds, test_ids, "nn_predictions.csv")